In [1]:
!pip install torch
!pip install torchvision
!pip install diffusers
!pip install tqdm
!pip install einops
!pip install timm


[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     --------------- ------------------------ 0.4/1.1 MB 13.5 MB/s eta 0:00:01
     --------------- ------------------------ 0.4/1.1 MB 13.5 MB/s eta 0:00:01
     --------------- ------------------------ 0.4/1.1 MB 13.5 MB/s eta 0:00:01
     --------------- ------------------------ 0.4/1.1 MB 13.5 MB/s eta 0:00:01
     --------------- ------------------------ 0.4/1.1 MB 13.5 MB/s eta 0:00:01
     --------------- ------------------------ 0.4/1.1 MB 13.5 MB/s eta 0:00:01
     -------------------- ------------------- 0.6/1.1 MB 1.7 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 3.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/44.6 kB ? eta -:--:--
     ---------------------------------------- 44.6/44.6 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
     -------- ------------------------------- 0.5/2.2 MB 9.4 MB/s eta 0:00:01
     ------------------- -------------------- 1.1/2.2 MB 11.3 MB/s eta 0:00:01
     ------------------------------- -------- 1.7/2.2 MB 12.2 MB/s eta 0:00:01
     ---------------------------------------- 2.2/2.2 MB 11.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip


In [2]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
from diffusion import create_diffusion
from diffusers.models import AutoencoderKL
from download import find_model
from FGFormer.FGFormer import FGFormers
import random

c:\Users\11429\anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
torch.manual_seed(0)
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [5]:
model = FGFormers["FGFormer-B/4"](
        input_size=32,
        num_classes=14
).to(device)
ckpt_path = "0090000.pt"
state_dict = find_model(ckpt_path)
model.load_state_dict(state_dict)
model.eval()
diffusion = create_diffusion(str(1000))
vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-mse").to(device)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
config.json: 100%|██████████| 547/547 [00:00<?, ?B/s] 
c:\Users\11429\anaconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\11429\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Devel

In [6]:
class_labels = [random.randint(0, 14 - 1) for _ in range(8)]
n = len(class_labels)
z = torch.randn(n, 4, 32, 32, device=device)
y = torch.tensor(class_labels, device=device)

In [7]:
z = torch.cat([z, z], 0)
y_null = torch.tensor([14] * n, device=device)
y = torch.cat([y, y_null], 0)
model_kwargs = dict(y=y, cfg_scale=4.0)

In [8]:
samples = diffusion.p_sample_loop(
        model.forward_with_cfg, z.shape, z, clip_denoised=False, model_kwargs=model_kwargs, progress=True, device=device
    )
samples, _ = samples.chunk(2, dim=0)
samples = vae.decode(samples / 0.18215).sample

  5%|▌         | 52/1000 [00:40<12:21,  1.28it/s]


KeyboardInterrupt: 

In [ ]:
samples